# Robust non-linear regression on a Gaussian dataset

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
import numpy as np
import math
import copy
import random
from statistics import mean
from my_optimizers import GD, Adam, Cata
from copy import deepcopy
%config InlineBackend.figure_format = 'svg'
plt.rcParams.update({'font.size': 14})
torch.set_deterministic(True)
torch.set_default_dtype(torch.float64)
import matplotlib

### Parameters

In [ ]:
lam = 1.005
n = 1000
bs = 1000
n_batches = math.floor(n/bs)
d = 500
nit = 2000
nexp = 3

### Data Generation

In [ ]:
seed = 7
np.random.seed(seed)
A = np.random.multivariate_normal(np.zeros(d), np.eye(d), n)
x_star = np.random.normal(0, 1, size=d)
y_0 = A @ x_star + np.random.normal(0, 100, size=n)
y_0 = y_0.reshape(-1,1)
A = torch.tensor(A, requires_grad=False)

### Model Definition

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden_1=256, n_hidden_2=64, n_output=1):
        super(Net, self).__init__()
        self.hidden_1 = torch.nn.Linear(n_feature, n_hidden_1)
        self.hidden_2 = torch.nn.Linear(n_hidden_1, n_hidden_2)
        self.predict = torch.nn.Linear(n_hidden_2, n_output)

    def forward(self, x):
        x = F.relu(self.hidden_1(x))
        x = F.relu(self.hidden_2(x))
        x = self.predict(x)             
        return x

### Training Function

In [ ]:
def train(settings):
    
    ### Init history variables
    grad_x_list = torch.zeros((nit,nexp), requires_grad=False)
    grad_y_list = torch.zeros((nit,nexp), requires_grad=False)
    
    for e in range(nexp):
        
        ### Init the model
        random_state=123
        np.random.seed(random_state)
        torch.manual_seed(random_state)
        net = Net(n_feature=d)
        criterion = torch.nn.MSELoss()    
        y = y_0 + np.random.normal(0, 10, size=(n,1))
        y = torch.tensor(y, requires_grad=False)

        ### Init optimizers
        if settings["optim"] == 'AGDA':
            tau_1=settings["tau1"]
            tau_2=settings["tau2"]
            optimizer_x = GD(net.parameters(), lr=tau_1)
            name = r""+settings["optim"]

        elif settings["optim"] == 'Smooth-AGDA':
            tau_1 = settings["tau1"]
            tau_2 = settings["tau2"]
            beta_s = settings["beta"]
            P_s = settings["P"]            
            optimizer_x = Cata(net.parameters(), lr=tau_1, beta = beta_s, P = P_s)
            name = r""+settings["optim"]
        else:
            print('ERROR, optimizer not defined')

        ### Optimization
        for i in range(nit):
            
            with torch.no_grad():
                # Sampling batches for this iteration
                number_list = range(n)
                batch=np.array(random.sample(number_list, bs))

                # Data for this batch
                X_iter = A[batch,:]
                y_iter = y[batch]
                y_0_iter = y_0[batch]
                
            # update for y
            with torch.no_grad():
                output_ = net(X_iter).detach().numpy()    
                y_iter_ = y_iter.detach().numpy()        
                grad_y = 2 * (y_iter_ - output_ - lam * (y_iter_ - y_0_iter)) / bs
                y_iter_ = y_iter_ + tau_2 * grad_y
                y_ = y.detach().numpy() 
                y_[batch] = y_iter_
                y = torch.tensor(y_, requires_grad=False)
                
            # update for x
            net.train()
            output = net(X_iter)
            loss = criterion(output, y_iter)
            def closure_x():
                optimizer_x.zero_grad()
                loss.backward()
            optimizer_x.step(closure_x)

            
            # compute and store gradient norms
            net.zero_grad()
            X_eval = copy.deepcopy(A[batch,:])
            X_eval.requires_grad = True
            y_eval = copy.deepcopy(y[batch])
            y_eval.requires_grad = True
            y_0_eval = y_0[batch]
            output = net(X_eval)
            loss = criterion(output, y_eval)
            loss.backward()
            with torch.no_grad():
                grad_y = 2 * (y_eval.detach().numpy() - output.detach().numpy() - lam * (y_eval.detach().numpy() - y_0_eval)) / n
                grad_x = [p.grad.data.detach().numpy() for p in net.parameters()]
                grad_x = np.concatenate(grad_x, axis=None)
                grad_x_list[i,e]=np.linalg.norm(grad_x)
                grad_y_list[i,e]=np.linalg.norm(grad_y)
        
    return name, grad_x_list, grad_y_list

### Comparisons

In [ ]:
### Definition of experiments
to_run=[]
to_run.append({"optim":"AGDA", "tau1": 0.0005, "tau2": 5})
to_run.append({"optim":"Smooth-AGDA", "tau1": 0.0005, "tau2": 5, "beta":0.5, "P":20})

### Running the experiments
gx = []
gy = []
names = []
for i in range(len(to_run)):
    names_c, gx_c, gy_c = train(to_run[i])
    names.append(names_c)
    gx.append(gx_c)
    gy.append(gy_c)

### Plot

In [ ]:
scale = 0.8
plt.rcParams["figure.figsize"] = (12,5)

markers = ["v","^","<",">","o","s","p","P","*"]
colors = sns.color_palette()

for i in range(len(to_run)):
    mean_x_log = np.mean(np.log10(gx[i].detach().numpy()),1)
    std_x_log = np.std(np.log10(gx[i].detach().numpy()), 1)

    mean_y_log = np.mean(np.log10(gy[i].detach().numpy()),1)
    std_y_log = np.std(np.log10(gy[i].detach().numpy()), 1)
    
    mean_x_log_s = savgol_filter(mean_x_log, 301, 3, mode='nearest')
    mean_y_log_s = savgol_filter(mean_y_log, 301, 3, mode='nearest')

    std_x_log_s = savgol_filter(std_x_log, 301, 3, mode='nearest')
    std_y_log_s = savgol_filter(std_y_log, 301, 3, mode='nearest')  
    
    mean_log_s = mean_x_log_s+mean_y_log_s
    std_log_s = std_x_log_s+std_y_log_s

    ax = plt.subplot(122)
    plt.plot(range(nit),np.power(10,mean_log_s), marker = markers[i%7], label=names[i],linewidth=3,color =  colors[i%10], markevery=500, markersize = 12)
    plt.fill_between(range(nit),np.power(10,mean_log_s-scale*std_log_s) , np.power(10,mean_log_s+scale*std_log_s), alpha=0.5, fc= colors[i%10])
    plt.xlabel("Iterations")
    plt.ylabel(r'$\Vert\nabla_x F(x,y)\Vert+\Vert\nabla_y F(x,y)\Vert$')
    plt.title(r'Synthetic Dataset, bs = 1000')
    plt.yscale("log")
    plt.legend()
    ax.yaxis.tick_right()
    plt.grid()
    